<a href="https://colab.research.google.com/github/NipunaCoder97/Data-Cleaning-Sales-Data/blob/main/Data_Cleaning_Sales_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

Import Datasets

In [ ]:
#Separately import datasets

actual_matrix=pd.read_csv("Dataset/actual_matrix.csv")
catalog=pd.read_csv("Dataset/catalog.csv")
discounts_history=pd.read_csv("Dataset/discounts_history.csv")
markdowns=pd.read_csv("Dataset/markdowns.csv")
online=pd.read_csv("Dataset/online.csv")
price_history=pd.read_csv("Dataset/price_history.csv")
sales=pd.read_csv("Dataset/sales.csv")
sample_submission=pd.read_csv("Dataset/sample_submission.csv")
store=pd.read_csv("Dataset/stores.csv")
test=pd.read_csv("Dataset/test.csv",sep=';')

Identify the test data set features

In [ ]:
test.head()
#The data frame has {"row_id, item_id, store_id, date"}. According to the test data set there are several items with store id and the sale date.
#whe predicting the demand according to the product, the store id will not be a feature impact on the model.
#If we group this data set according to the dataset, this will be univariate time series forecasting approach.
#The model need to predict the quantity of the item need in future

,row_id,item_id,store_id,date
0,0,c578da8e8841,1,27.09.2024
1,1,c578da8e8841,1,28.09.2024
2,2,c578da8e8841,1,29.09.2024
3,3,c578da8e8841,1,30.09.2024
4,4,c578da8e8841,1,01.10.2024


In [ ]:
online.columns

Index(['Unnamed: 0', 'date', 'item_id', 'quantity', 'price_base', 'sum_total',
       'store_id'],
      dtype='object')

In [ ]:
sales.columns

Index(['Unnamed: 0', 'date', 'item_id', 'quantity', 'price_base', 'sum_total',
       'store_id'],
      dtype='object')

The forecasting consider only the amount of the items need in future. Therefore, no need to forecast under different buckets such as online and sale buckets. Therefore, I merge two dataframes into one dataframe to develop the model.

In [ ]:
sales.shape, online.shape

((7432685, 7), (1123412, 7))

In [ ]:
df_combined = pd.concat([online, sales], ignore_index=True)

In [ ]:
df_combined.shape

(8556097, 7)

In [ ]:
Item_List = list(df_combined["item_id"].unique())

In [ ]:
len(Item_List)

28323

In [ ]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8556097 entries, 0 to 8556096
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   date        object 
 2   item_id     object 
 3   quantity    float64
 4   price_base  float64
 5   sum_total   float64
 6   store_id    int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 456.9+ MB


In [ ]:
df_combined.isnull().sum()

Unnamed: 0    0
date          0
item_id       0
quantity      0
price_base    0
sum_total     0
store_id      0
dtype: int64

In [ ]:
train_df = df_combined[["date","item_id","store_id","quantity"]]

In [ ]:
train_df.tail(), train_df.head()

#The data set has data from 2023-08-04 to 2024-09-26

(               date       item_id  store_id  quantity
 8556092  2024-09-26  542f4875a846         4       2.0
 8556093  2024-09-26  3c7435256b1f         4       2.0
 8556094  2024-09-26  87b794255afc         4       6.0
 8556095  2024-09-26  0a832c545a6a         4       1.0
 8556096  2024-09-26  c304e866ad10         4       1.0,
          date       item_id  store_id  quantity
 0  2023-08-04  4aa8dbe05246         1       3.0
 1  2023-08-04  4e0fbcf99cf9         1       2.0
 2  2023-08-04  2e008b673129         1       2.0
 3  2023-08-04  49db9c973903         1       1.0
 4  2023-08-04  9d6072422e8e         1       1.0)

Now need to transform data into relavent data types

In [ ]:
train_df["date"] = pd.to_datetime(train_df["date"])
test['date'] = pd.to_datetime(test['date'])

C:\Users\asus\AppData\Local\Temp\ipykernel_15680\1999329847.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["date"] = pd.to_datetime(train_df["date"])
C:\Users\asus\AppData\Local\Temp\ipykernel_15680\1999329847.py:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  test['date'] = pd.to_datetime(test['date'])


In [ ]:
train_df["year"] = train_df['date'].dt.year
train_df["month"] = train_df['date'].dt.month
train_df["day"] = train_df['date'].dt.day
train_df.drop(columns=['date'], inplace=True)


test["year"] = test['date'].dt.year
test["month"] = test['date'].dt.month
test["day"] = test['date'].dt.day
test.drop(columns=['date'], inplace=True)


C:\Users\asus\AppData\Local\Temp\ipykernel_15680\2250947963.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["year"] = train_df['date'].dt.year
C:\Users\asus\AppData\Local\Temp\ipykernel_15680\2250947963.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df["month"] = train_df['date'].dt.month


In [ ]:
df = train_df

In [ ]:

df["day_sin"] = np.sin(2 * np.pi * df["day"] / 31)
df["day_cos"] = np.cos(2 * np.pi * df["day"] / 31)
df["month_sin"] = np.sin(2 * np.pi * df["month"] / 12)
df["month_cos"] = np.cos(2 * np.pi * df["month"] / 12)

test["day_sin"] = np.sin(2 * np.pi * test["day"] / 31)
test["day_cos"] = np.cos(2 * np.pi * test["day"] / 31)
test["month_sin"] = np.sin(2 * np.pi * test["month"] / 12)
test["month_cos"] = np.cos(2 * np.pi * test["month"] / 12)


In [ ]:
df.drop(df[df['quantity']<0].index, inplace=True)
df.drop(columns = ['month', 'day'], inplace=True)
test.drop(columns = ['month', 'day'], inplace=True)
test.drop(columns=['row_id'],inplace=True)

In [ ]:
df=df[['item_id','store_id','year','day_sin','day_cos','month_sin','month_cos','quantity']]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

columnstoscale = ['store_id','year']
df[columnstoscale] = scaler.fit_transform(df[columnstoscale])

columnstoscaletest = ['store_id','year']
test[columnstoscaletest] = scaler.transform(test[columnstoscaletest])

In [ ]:
qty = ['quantity']
df[qty] = scaler.fit_transform(df[qty])

In [ ]:
df.head()

,item_id,store_id,year,day_sin,day_cos,month_sin,month_cos,quantity
0,4aa8dbe05246,-0.813659,-0.514157,0.724793,0.688967,-0.866025,-0.5,-0.085601
1,4e0fbcf99cf9,-0.813659,-0.514157,0.724793,0.688967,-0.866025,-0.5,-0.124657
2,2e008b673129,-0.813659,-0.514157,0.724793,0.688967,-0.866025,-0.5,-0.124657
3,49db9c973903,-0.813659,-0.514157,0.724793,0.688967,-0.866025,-0.5,-0.163713
4,9d6072422e8e,-0.813659,-0.514157,0.724793,0.688967,-0.866025,-0.5,-0.163713


In [ ]:
test.head()

,item_id,store_id,year,day_sin,day_cos,month_sin,month_cos
0,c578da8e8841,-0.813659,0.91972,-0.724793,0.688967,-1.0,-1.836970e-16
1,c578da8e8841,-0.813659,0.91972,-0.571268,0.820763,-1.0,-1.836970e-16
2,c578da8e8841,-0.813659,0.91972,-0.394356,0.918958,-1.0,-1.836970e-16
3,c578da8e8841,-0.813659,0.91972,-0.201299,0.979530,-1.0,-1.836970e-16
4,c578da8e8841,-0.813659,0.91972,0.897805,-0.440394,0.5,8.660254e-01


In [ ]:
# Save train and test DataFrames to CSV
df.to_csv("cleaned_train.csv", index=False)  # Saves without the index column
test.to_csv("cleaned_test.csv", index=False)

print("Train and Test datasets saved successfully!")


✅ Train and Test datasets saved successfully!
